In [1]:
import numpy as np
import pandas as pd

In [2]:
# 1. 生成1项集
# 2. 生成

In [3]:
def create_C1(data_set):
    C1 = set()
    for t in data_set:
        for item in t:
            item_set = frozenset([item])
            C1.add(item_set)
    return C1

In [4]:
def is_apriori(Ck_item, Lksub1):
    for item in Ck_item:
        sub_Ck = Ck_item - frozenset([item])
        if sub_Ck not in Lksub1:
            return False
    return True

In [5]:
def create_Ck(Lksub1, k):
    Ck = set()
    len_Lksub1 = len(Lksub1)
    list_Lksub1 = list(Lksub1)
    print(list_Lksub1)
    for i in range(len_Lksub1):
        for j in range(i, len_Lksub1):
            l1 = list(list_Lksub1[i])
            l2 = list(list_Lksub1[j])
            l1.sort()
            l2.sort()
            if l1[0:k-2] == l2[0:k-2]:
                Ck_item = list_Lksub1[i] | list_Lksub1[j]
                # pruning
                if is_apriori(Ck_item, Lksub1):
                    Ck.add(Ck_item)
    return Ck

In [6]:
def generate_Lk_by_Ck(data_set, Ck, min_support, support_data):
    Lk = set()
    item_count = {}
    for t in data_set:
        for item in Ck:
            if item.issubset(t):
                if item not in item_count:
                    item_count[item] = 1
                else:
                    item_count[item] += 1
    t_num = float(len(data_set))
    for item in item_count:
        if (item_count[item] / t_num) >= min_support:
            Lk.add(item)
            support_data[item] = item_count[item] / t_num
    return Lk


In [7]:
# 生成频繁项集
def generate_L(data_set, k, min_support):
    support_data = {}
    C1 = create_C1(data_set)
    L1 = generate_Lk_by_Ck(data_set, C1, min_support, support_data)
    Lksub1 = L1.copy()
    L = []
    L.append(Lksub1)
    for i in range(2, k+1):
        Ci = create_Ck(Lksub1, i)
        Li = generate_Lk_by_Ck(data_set, Ci, min_support, support_data)
        Lksub1 = Li.copy()
        L.append(Lksub1)
    return L, support_data

In [8]:
# 生成关联规则
def generate_big_rules(L, support_data, min_conf):
    big_rule_list = []
    sub_set_list = []
    for i in range(0, len(L)):
        for freq_set in L[i]:
            for sub_set in sub_set_list:
                if sub_set.issubset(freq_set):
                    conf = support_data[freq_set] / support_data[freq_set - sub_set]
                    big_rule = (freq_set - sub_set, sub_set, conf)
                    if conf >= min_conf and big_rule not in big_rule_list:
                        # print freq_set-sub_set, " => ", sub_set, "conf: ", conf
                        big_rule_list.append(big_rule)
            sub_set_list.append(freq_set)
    return big_rule_list

In [9]:
if __name__ == "__main__":
    
    # 获取并处理数据集，去除空值
    data = pd.read_csv("goods2.csv",header = None)
    data = np.array(data)
    data_set = [[i for i in row if not pd.isna(i)] for row in data]
    
    # 生成频繁项集
    L, support_data = generate_L(data_set, k=3, min_support=0.5)
    
    # 生成关联规则
    big_rules_list = generate_big_rules(L, support_data, min_conf=0.7)
    
    for Lk in L:
        print("="*50)
        print("frequent " + str(len(list(Lk)[0])) + "-itemsets\t\tsupport")
        print("="*50)
        for freq_set in Lk:
            print(freq_set, support_data[freq_set])
    print ("Big Rules")
    for item in big_rules_list:
        print(item[0], "=>", item[1], "conf: ", item[2])

[frozenset({'good2'}), frozenset({'good3'}), frozenset({'good5'}), frozenset({'good1'})]
[frozenset({'good2', 'good5'}), frozenset({'good2', 'good3'}), frozenset({'good3', 'good1'}), frozenset({'good3', 'good5'})]
frequent 1-itemsets		support
frozenset({'good2'}) 0.75
frozenset({'good3'}) 0.75
frozenset({'good5'}) 0.75
frozenset({'good1'}) 0.5
frequent 2-itemsets		support
frozenset({'good2', 'good5'}) 0.75
frozenset({'good2', 'good3'}) 0.5
frozenset({'good3', 'good1'}) 0.5
frozenset({'good3', 'good5'}) 0.5
frequent 3-itemsets		support
frozenset({'good2', 'good3', 'good5'}) 0.5
Big Rules
frozenset({'good5'}) => frozenset({'good2'}) conf:  1.0
frozenset({'good2'}) => frozenset({'good5'}) conf:  1.0
frozenset({'good1'}) => frozenset({'good3'}) conf:  1.0
frozenset({'good3', 'good5'}) => frozenset({'good2'}) conf:  1.0
frozenset({'good2', 'good3'}) => frozenset({'good5'}) conf:  1.0
